# NAO Deep Learning Custom model training (based on Tiny YOLOv4)

Welcome to the Deep NAO custom model training tutorial. In this notebook we demonstrate how to train your custom object detection deep learning model to use it in the Deep NAO application.

The model you will train uses the Tiny YOLOv4 architecture. This Colab takes around 1h30 to complete. 

## Choose GPU for your Colab instance 

To execute this notebook, we recommend that you Click on `Edit` > `Notebook settings` in the toolbar of this page, and select `GPU` in the field `Hardware`

## Retrieve and compile the Darknet framework

[Darknet](https://pjreddie.com/darknet/) is the Deep Learning framework that you will use to train your custom model. Your custom model will use a Tiny YOLOv4 architecture.

Download darknet and uncompress it.

In [ ]:
# Download Darknet source code from the Github repository
!wget https://github.com/AlexeyAB/darknet/archive/darknet_yolo_v4_pre.zip
# Extract the archive
!unzip darknet_yolo_v4_pre.zip
# Rename the folder as 'darknet'
%mv darknet-darknet_yolo_v4_pre darknet


Modify the Makefile to enable GPU (this will run faster) and OpenCV. Then compile:

In [ ]:
# Go to 'darknet' folder
%cd darknet
# Edit the Makefile to enable OpenCV, GPU, CUDNN and CUDNN_HALF
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile
# Compile the Darknet framework code
!make -j2

Now that the Darknet framework is installed, let's get some training dataset.

## Prepare your training data

### Google Open Image Dataset

To train our deep learning model, we need a dataset of images containing the objects we want to detect. The images must be labeled. Which means for each image, if it contains the object to be detected, we need to provide to the training algorithm the coordinates of the object in the image. 

Fortunately for us, there exists datasets of already labelled images out there on the internet. Google provides such a dataset: the [Open Image Dataset](https://storage.googleapis.com/openimages/web/index.html), that contains thousand of labelled images for 600 classes of objects.

You can explore the dataset using the [Visualizer](https://storage.googleapis.com/openimages/web/visualizer/index.html?set=train&type=detection&c=%2Fm%2F01lsmm) they provide.


### Download labeled object images from Google Open Image Dataset

In this tutorial we train our model to recognize 2 different objects out of the 600 objects available in the Google Image dataset. Choose below which objects you want nao to learn to recognize, and launch the download of labeled images of these objects.

In [ ]:
#@title Choose the objects NAO will learn to recognize

first_object = 'Glasses' #@param ["Accordion", "Adhesive tape", "Aircraft", "Airplane", "Alarm clock", "Alpaca", "Ambulance", "Animal", "Ant", "Antelope", "Apple", "Armadillo", "Artichoke", "Asparagus", "Auto part", "Axe", "Backpack", "Bagel", "Baked goods", "Balance beam", "Ball", "Balloon", "Banana", "Band-aid", "Banjo", "Barge", "Barrel", "Baseball bat", "Baseball glove", "Bat", "Bathroom accessory", "Bathroom cabinet", "Bathtub", "Beaker", "Bear", "Bed", "Bee", "Beehive", "Beer", "Beetle", "Bell pepper", "Belt", "Bench", "Bicycle", "Bicycle helmet", "Bicycle wheel", "Bidet", "Billboard", "Billiard table", "Binoculars", "Bird", "Blender", "Blue jay", "Boat", "Bomb", "Book", "Bookcase", "Boot", "Bottle", "Bottle opener", "Bow and arrow", "Bowl", "Bowling equipment", "Box", "Boy", "Brassiere", "Bread", "Briefcase", "Broccoli", "Bronze sculpture", "Brown bear", "Building", "Bull", "Burrito", "Bus", "Bust", "Butterfly", "Cabbage", "Cabinetry", "Cake", "Cake stand", "Calculator", "Camel", "Camera", "Canary", "Candle", "Candy", "Cannon", "Canoe", "Can opener", "Cantaloupe", "Car", "Carnivore", "Carrot", "Cart", "Cassette deck", "Castle", "Cat", "Caterpillar", "Cat furniture", "Cattle", "Ceiling fan", "Cello", "Centipede", "Chainsaw", "Chair", "Cheese", "Cheetah", "Chest of drawers", "Chicken", "Chime", "Chisel", "Chopsticks", "Christmas tree", "Clock", "Closet", "Clothing", "Coat", "Cocktail", "Cocktail shaker", "Coconut", "Coffee", "Coffee cup", "Coffeemaker", "Coffee table", "Coin", "Common fig", "Computer keyboard", "Computer monitor", "Computer mouse", "Container", "Convenience store", "Cookie", "Cooking spray", "Corded phone", "Cosmetics", "Couch", "Countertop", "Cowboy hat", "Crab", "Cream", "Cricket ball", "Crocodile", "Croissant", "Crown", "Crutch", "Cucumber", "Cupboard", "Curtain", "Cutting board", "Dagger", "Dairy", "Deer", "Desk", "Dessert", "Diaper", "Dice", "Digital clock", "Dinosaur", "Dishwasher", "Dog", "Dog bed", "Doll", "Dolphin", "Door", "Door handle", "Doughnut", "Dragonfly", "Drawer", "Dress", "Drill", "Drink", "Drinking straw", "Drum", "Duck", "Dumbbell", "Eagle", "Earrings", "Egg", "Elephant", "Envelope", "Eraser", "Face powder", "Facial tissue holder", "Falcon", "Fashion accessory", "Fast food", "Fax", "Fedora", "Filing cabinet", "Fire hydrant", "Fireplace", "Fish", "Flag", "Flashlight", "Flower", "Flowerpot", "Flute", "Flying disc", "Food", "Food processor", "Football", "Football helmet", "Footwear", "Fork", "Fountain", "Fox", "French fries", "Frog", "Fruit", "Frying pan", "Furniture", "Gas stove", "Giraffe", "Girl", "Glasses", "Glove", "Goat", "Goggles", "Goldfish", "Golf ball", "Golf cart", "Gondola", "Goose", "Grape", "Grapefruit", "Grinder", "Guacamole", "Guitar", "Hair dryer", "Hair spray", "Hamburger", "Hammer", "Hamster", "Handbag", "Hand dryer", "Handgun", "Harbor seal", "Harmonica", "Harp", "Harpsichord", "Hat", "Headphones", "Heater", "Hedgehog", "Helicopter", "Helmet", "High heels", "Hiking equipment", "Hippopotamus", "Home appliance", "Honeycomb", "Horizontal bar", "Horn", "Horse", "Hot dog", "House", "Houseplant", "Human arm", "Human beard", "Human body", "Human ear", "Human eye", "Human face", "Human foot", "Human hair", "Human hand", "Human head", "Human leg", "Human mouth", "Human nose", "Humidifier", "Ice cream", "Indoor rower", "Infant bed", "Insect", "Invertebrate", "Ipod", "Isopod", "Jacket", "Jacuzzi", "Jaguar", "Jeans", "Jellyfish", "Jet ski", "Jug", "Juice", "Kangaroo", "Kettle", "Kitchen appliance", "Kitchen & dining room table", "Kitchen knife", "Kitchen utensil", "Kitchenware", "Kite", "Knife", "Koala", "Ladder", "Ladle", "Ladybug", "Lamp", "Land vehicle", "Lantern", "Laptop", "Lavender", "Lemon", "Leopard", "Lifejacket", "Light bulb", "Lighthouse", "Light switch", "Lily", "Limousine", "Lion", "Lipstick", "Lizard", "Lobster", "Loveseat", "Luggage and bags", "Lynx", "Magpie", "Mammal", "Man", "Mango", "Maple", "Maracas", "Marine invertebrates", "Marine mammal", "Measuring cup", "Mechanical fan", "Medical equipment", "Microphone", "Microwave oven", "Milk", "Miniskirt", "Mirror", "Missile", "Mixer", "Mixing bowl", "Mobile phone", "Monkey", "Moths and butterflies", "Motorcycle", "Mouse", "Muffin", "Mug", "Mule", "Mushroom", "Musical instrument", "Musical keyboard", "Nail", "Necklace", "Nightstand", "Oboe", "Office building", "Office supplies", "Orange", "Organ", "Ostrich", "Otter", "Oven", "Owl", "Oyster", "Paddle", "Palm tree", "Pancake", "Panda", "Paper cutter", "Paper towel", "Parachute", "Parking meter", "Parrot", "Pasta", "Pastry", "Peach", "Pear", "Pen", "Pencil case", "Pencil sharpener", "Penguin", "Perfume", "Person", "Personal care", "Piano", "Picnic basket", "Picture frame", "Pig", "Pillow", "Pineapple", "Pitcher", "Pizza", "Pizza cutter", "Plant", "Plastic bag", "Plate", "Platter", "Plumbing fixture", "Polar bear", "Pomegranate", "Popcorn", "Porch", "Porcupine", "Poster", "Potato", "Power plugs and sockets", "Pressure cooker", "Pretzel", "Printer", "Pumpkin", "Punching bag", "Rabbit", "Raccoon", "Racket", "Radish", "Ratchet", "Raven", "Rays and skates", "Red panda", "Refrigerator", "Remote control", "Reptile", "Rhinoceros", "Rifle", "Ring binder", "Rocket", "Roller skates", "Rose", "Rugby ball", "Ruler", "Salad", "Salt and pepper shakers", "Sandal", "Sandwich", "Saucer", "Saxophone", "Scale", "Scarf", "Scissors", "Scoreboard", "Scorpion", "Screwdriver", "Sculpture", "Seafood", "Seahorse", "Sea lion", "Seat belt", "Sea turtle", "Segway", "Serving tray", "Sewing machine", "Shark", "Sheep", "Shelf", "Shellfish", "Shirt", "Shorts", "Shotgun", "Shower", "Shrimp", "Sink", "Skateboard", "Ski", "Skirt", "Skull", "Skunk", "Skyscraper", "Slow cooker", "Snack", "Snail", "Snake", "Snowboard", "Snowman", "Snowmobile", "Snowplow", "Soap dispenser", "Sock", "Sofa bed", "Sombrero", "Sparrow", "Spatula", "Spice rack", "Spider", "Spoon", "Sports equipment", "Sports uniform", "Squash", "Squid", "Squirrel", "Stairs", "Stapler", "Starfish", "Stationary bicycle", "Stethoscope", "Stool", "Stop sign", "Strawberry", "Street light", "Stretcher", "Studio couch", "Submarine", "Submarine sandwich", "Suit", "Suitcase", "Sunflower", "Sunglasses", "Sun hat", "Surfboard", "Sushi", "Swan", "Swim cap", "Swimming pool", "Swimwear", "Sword", "Syringe", "Table", "Tablet computer", "Table tennis racket", "Tableware", "Taco", "Tank", "Tap", "Tart", "Taxi", "Tea", "Teapot", "Teddy bear", "Telephone", "Television", "Tennis ball", "Tennis racket", "Tent", "Tiara", "Tick", "Tie", "Tiger", "Tin can", "Tire", "Toaster", "Toilet", "Toilet paper", "Tomato", "Tool", "Toothbrush", "Torch", "Tortoise", "Towel", "Tower", "Toy", "Traffic light", "Traffic sign", "Train", "Training bench", "Treadmill", "Tree", "Tree house", "Tripod", "Trombone", "Trousers", "Truck", "Trumpet", "Turkey", "Turtle", "Umbrella", "Unicycle", "Van", "Vase", "Vegetable", "Vehicle", "Vehicle registration plate", "Violin", "Volleyball", "Waffle", "Waffle iron", "Wall clock", "Wardrobe", "Washing machine", "Waste container", "Watch", "Watercraft", "Watermelon", "Weapon", "Whale", "Wheel", "Wheelchair", "Whisk", "Whiteboard", "Willow", "Window", "Window blind", "Wine", "Wine glass", "Wine rack", "Winter melon", "Wok", "Woman", "Wood-burning stove", "Woodpecker", "Worm", "Wrench", "Zebra", "Zucchini"]
second_object = 'Mobile phone'  #@param ["Accordion", "Adhesive tape", "Aircraft", "Airplane", "Alarm clock", "Alpaca", "Ambulance", "Animal", "Ant", "Antelope", "Apple", "Armadillo", "Artichoke", "Asparagus", "Auto part", "Axe", "Backpack", "Bagel", "Baked goods", "Balance beam", "Ball", "Balloon", "Banana", "Band-aid", "Banjo", "Barge", "Barrel", "Baseball bat", "Baseball glove", "Bat", "Bathroom accessory", "Bathroom cabinet", "Bathtub", "Beaker", "Bear", "Bed", "Bee", "Beehive", "Beer", "Beetle", "Bell pepper", "Belt", "Bench", "Bicycle", "Bicycle helmet", "Bicycle wheel", "Bidet", "Billboard", "Billiard table", "Binoculars", "Bird", "Blender", "Blue jay", "Boat", "Bomb", "Book", "Bookcase", "Boot", "Bottle", "Bottle opener", "Bow and arrow", "Bowl", "Bowling equipment", "Box", "Boy", "Brassiere", "Bread", "Briefcase", "Broccoli", "Bronze sculpture", "Brown bear", "Building", "Bull", "Burrito", "Bus", "Bust", "Butterfly", "Cabbage", "Cabinetry", "Cake", "Cake stand", "Calculator", "Camel", "Camera", "Canary", "Candle", "Candy", "Cannon", "Canoe", "Can opener", "Cantaloupe", "Car", "Carnivore", "Carrot", "Cart", "Cassette deck", "Castle", "Cat", "Caterpillar", "Cat furniture", "Cattle", "Ceiling fan", "Cello", "Centipede", "Chainsaw", "Chair", "Cheese", "Cheetah", "Chest of drawers", "Chicken", "Chime", "Chisel", "Chopsticks", "Christmas tree", "Clock", "Closet", "Clothing", "Coat", "Cocktail", "Cocktail shaker", "Coconut", "Coffee", "Coffee cup", "Coffeemaker", "Coffee table", "Coin", "Common fig", "Computer keyboard", "Computer monitor", "Computer mouse", "Container", "Convenience store", "Cookie", "Cooking spray", "Corded phone", "Cosmetics", "Couch", "Countertop", "Cowboy hat", "Crab", "Cream", "Cricket ball", "Crocodile", "Croissant", "Crown", "Crutch", "Cucumber", "Cupboard", "Curtain", "Cutting board", "Dagger", "Dairy", "Deer", "Desk", "Dessert", "Diaper", "Dice", "Digital clock", "Dinosaur", "Dishwasher", "Dog", "Dog bed", "Doll", "Dolphin", "Door", "Door handle", "Doughnut", "Dragonfly", "Drawer", "Dress", "Drill", "Drink", "Drinking straw", "Drum", "Duck", "Dumbbell", "Eagle", "Earrings", "Egg", "Elephant", "Envelope", "Eraser", "Face powder", "Facial tissue holder", "Falcon", "Fashion accessory", "Fast food", "Fax", "Fedora", "Filing cabinet", "Fire hydrant", "Fireplace", "Fish", "Flag", "Flashlight", "Flower", "Flowerpot", "Flute", "Flying disc", "Food", "Food processor", "Football", "Football helmet", "Footwear", "Fork", "Fountain", "Fox", "French fries", "Frog", "Fruit", "Frying pan", "Furniture", "Gas stove", "Giraffe", "Girl", "Glasses", "Glove", "Goat", "Goggles", "Goldfish", "Golf ball", "Golf cart", "Gondola", "Goose", "Grape", "Grapefruit", "Grinder", "Guacamole", "Guitar", "Hair dryer", "Hair spray", "Hamburger", "Hammer", "Hamster", "Handbag", "Hand dryer", "Handgun", "Harbor seal", "Harmonica", "Harp", "Harpsichord", "Hat", "Headphones", "Heater", "Hedgehog", "Helicopter", "Helmet", "High heels", "Hiking equipment", "Hippopotamus", "Home appliance", "Honeycomb", "Horizontal bar", "Horn", "Horse", "Hot dog", "House", "Houseplant", "Human arm", "Human beard", "Human body", "Human ear", "Human eye", "Human face", "Human foot", "Human hair", "Human hand", "Human head", "Human leg", "Human mouth", "Human nose", "Humidifier", "Ice cream", "Indoor rower", "Infant bed", "Insect", "Invertebrate", "Ipod", "Isopod", "Jacket", "Jacuzzi", "Jaguar", "Jeans", "Jellyfish", "Jet ski", "Jug", "Juice", "Kangaroo", "Kettle", "Kitchen appliance", "Kitchen & dining room table", "Kitchen knife", "Kitchen utensil", "Kitchenware", "Kite", "Knife", "Koala", "Ladder", "Ladle", "Ladybug", "Lamp", "Land vehicle", "Lantern", "Laptop", "Lavender", "Lemon", "Leopard", "Lifejacket", "Light bulb", "Lighthouse", "Light switch", "Lily", "Limousine", "Lion", "Lipstick", "Lizard", "Lobster", "Loveseat", "Luggage and bags", "Lynx", "Magpie", "Mammal", "Man", "Mango", "Maple", "Maracas", "Marine invertebrates", "Marine mammal", "Measuring cup", "Mechanical fan", "Medical equipment", "Microphone", "Microwave oven", "Milk", "Miniskirt", "Mirror", "Missile", "Mixer", "Mixing bowl", "Mobile phone", "Monkey", "Moths and butterflies", "Motorcycle", "Mouse", "Muffin", "Mug", "Mule", "Mushroom", "Musical instrument", "Musical keyboard", "Nail", "Necklace", "Nightstand", "Oboe", "Office building", "Office supplies", "Orange", "Organ", "Ostrich", "Otter", "Oven", "Owl", "Oyster", "Paddle", "Palm tree", "Pancake", "Panda", "Paper cutter", "Paper towel", "Parachute", "Parking meter", "Parrot", "Pasta", "Pastry", "Peach", "Pear", "Pen", "Pencil case", "Pencil sharpener", "Penguin", "Perfume", "Person", "Personal care", "Piano", "Picnic basket", "Picture frame", "Pig", "Pillow", "Pineapple", "Pitcher", "Pizza", "Pizza cutter", "Plant", "Plastic bag", "Plate", "Platter", "Plumbing fixture", "Polar bear", "Pomegranate", "Popcorn", "Porch", "Porcupine", "Poster", "Potato", "Power plugs and sockets", "Pressure cooker", "Pretzel", "Printer", "Pumpkin", "Punching bag", "Rabbit", "Raccoon", "Racket", "Radish", "Ratchet", "Raven", "Rays and skates", "Red panda", "Refrigerator", "Remote control", "Reptile", "Rhinoceros", "Rifle", "Ring binder", "Rocket", "Roller skates", "Rose", "Rugby ball", "Ruler", "Salad", "Salt and pepper shakers", "Sandal", "Sandwich", "Saucer", "Saxophone", "Scale", "Scarf", "Scissors", "Scoreboard", "Scorpion", "Screwdriver", "Sculpture", "Seafood", "Seahorse", "Sea lion", "Seat belt", "Sea turtle", "Segway", "Serving tray", "Sewing machine", "Shark", "Sheep", "Shelf", "Shellfish", "Shirt", "Shorts", "Shotgun", "Shower", "Shrimp", "Sink", "Skateboard", "Ski", "Skirt", "Skull", "Skunk", "Skyscraper", "Slow cooker", "Snack", "Snail", "Snake", "Snowboard", "Snowman", "Snowmobile", "Snowplow", "Soap dispenser", "Sock", "Sofa bed", "Sombrero", "Sparrow", "Spatula", "Spice rack", "Spider", "Spoon", "Sports equipment", "Sports uniform", "Squash", "Squid", "Squirrel", "Stairs", "Stapler", "Starfish", "Stationary bicycle", "Stethoscope", "Stool", "Stop sign", "Strawberry", "Street light", "Stretcher", "Studio couch", "Submarine", "Submarine sandwich", "Suit", "Suitcase", "Sunflower", "Sunglasses", "Sun hat", "Surfboard", "Sushi", "Swan", "Swim cap", "Swimming pool", "Swimwear", "Sword", "Syringe", "Table", "Tablet computer", "Table tennis racket", "Tableware", "Taco", "Tank", "Tap", "Tart", "Taxi", "Tea", "Teapot", "Teddy bear", "Telephone", "Television", "Tennis ball", "Tennis racket", "Tent", "Tiara", "Tick", "Tie", "Tiger", "Tin can", "Tire", "Toaster", "Toilet", "Toilet paper", "Tomato", "Tool", "Toothbrush", "Torch", "Tortoise", "Towel", "Tower", "Toy", "Traffic light", "Traffic sign", "Train", "Training bench", "Treadmill", "Tree", "Tree house", "Tripod", "Trombone", "Trousers", "Truck", "Trumpet", "Turkey", "Turtle", "Umbrella", "Unicycle", "Van", "Vase", "Vegetable", "Vehicle", "Vehicle registration plate", "Violin", "Volleyball", "Waffle", "Waffle iron", "Wall clock", "Wardrobe", "Washing machine", "Waste container", "Watch", "Watercraft", "Watermelon", "Weapon", "Whale", "Wheel", "Wheelchair", "Whisk", "Whiteboard", "Willow", "Window", "Window blind", "Wine", "Wine glass", "Wine rack", "Winter melon", "Wok", "Woman", "Wood-burning stove", "Woodpecker", "Worm", "Wrench", "Zebra", "Zucchini"]

# Download the OIDv4 toolkit from the Github repository. The OIDv4 toolkit 
# contains scripts to automate the download of images from the Google Open Image 
# Dataset
!git clone https://github.com/theAIGuysCode/OIDv4_ToolKit.git
# Go to the toolkit folder
%cd OIDv4_ToolKit

# Install python requirements
!pip install -r requirements.txt 

#Choose how many image of each object to use for the training (400 should be enough)
training_image_number = 400 #@param {type:"integer"}

# Download images of the objects you want to recognize from the Google Open Image Dataset
!python main.py downloader --classes "$first_object" "$second_object" --type_csv train --limit $training_image_number --yes

# Download 40 images for validation
!python main.py downloader --classes "$first_object" "$second_object"  --type_csv validation --limit 40 --yes

# Convert the images annotation files to YoloV4 format
!echo $first_object > classes.txt
!echo $second_object >> classes.txt 
!cat classes.txt
!python convert_annotations.py


Prepare the dataset for the training

In [ ]:
%%shell
# Create a data folder, containing an obj folder
mkdir -p data/obj
# Create the 'obj.names' file, that contains the names of the objects to recognize
cp classes.txt data/obj.names
# Create the 'obj.data' configuration file, used by Darknet
cat > data/obj.data <<EOF
classes = 2
train  = data/train.txt
valid  = data/test.txt
names = data/obj.names
backup = backup/
EOF
# Copy all the download images to the data/obj folder 
cp -r OID/Dataset/train/* data/obj/
# List all the images in the 'data/train.txt' file
find data/obj/ -name "*jpg" > data/train.txt
# Remove Darknet data folder 
rm -rf ../data
# And replace the Darknet data folder by ours
mv data ..

Resize all images to 416x416, the input size for the Tiny YOLOv4 model.

In [ ]:
# Install imagemagick
!sudo apt install imagemagick
# Resize all images to 416x416
!mogrify -verbose -strip -resize 416x416! -quality 75 data/obj/**/*jpg

That's it, our dataset is ready. Time to prepare the training of our model.

## Retrieve the deep learning model and setup the training

There are several architecture of deep learning networks that exists. We chose to use the Tiny YOLOv4 architecture, as it is fast enough and have a good enough acuracy. 

Retrieve the deep learning model from the Darknet github repository, and configure it to fit our objective to recognize two objects.

In [ ]:
# Go to Darknet folder
%cd /content/darknet
# Remove default config file 
%rm -f cfg/yolov4-tiny-custom.cfg
# Download custom config file
!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4-tiny-custom.cfg -P cfg
# Rename the config file to create ours
%cp cfg/yolov4-tiny-custom.cfg cfg/yolov4-tiny-tutorial.cfg
# Edit the config file as per darknet README recommendations, to train to recognize 2 classes
!sed -i '7s/subdivisions=1/subdivisions=16/' cfg/yolov4-tiny-tutorial.cfg
!sed -i '20s/max_batches = 500200/max_batches = 6001/' cfg/yolov4-tiny-tutorial.cfg
!sed -i '22s/steps=400000,450000/steps=4800,5400/' cfg/yolov4-tiny-tutorial.cfg
!sed -i '212s/filters=255/filters=21/' cfg/yolov4-tiny-tutorial.cfg
!sed -i '220s/classes=80/classes=2/' cfg/yolov4-tiny-tutorial.cfg
!sed -i '263s/filters=255/filters=21/' cfg/yolov4-tiny-tutorial.cfg
!sed -i '269s/classes=80/classes=2/' cfg/yolov4-tiny-tutorial.cfg

# Verify edition worked
!diff -u cfg/yolov4-tiny-custom.cfg cfg/yolov4-tiny-tutorial.cfg

We won't train the deep learning model weights from scratch. We will start with pretrained weights in order to speed up the training. We use weights pre-trained on the [Coco dataset](https://cocodataset.org/). Let's download them.





In [ ]:
# Remove default weights
%rm -f cfg/yolov4-tiny.conv.29 
# Download the pretrained weight
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29 -P cfg

## Mount your google drive account to backup weights

We use your google drive folder to save the model weights during the training. This will allow you to retrieve the weights at the end of the training. 

Execute the code below: it will tell you to open an url, log in your google drive, and  copy paste a secret code in the window below to allow to mount your google drive account. 

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount("drive")

# Create a backup directory in Google drive
%mkdir -p drive/MyDrive/NAO_DeepLearning/backup
# Remove the default 'backup' directory in Darknet
%rm -rf backup/
# Replace by ours: create a link to the backup directory in the Google drive
!ln -fs drive/MyDrive/NAO_DeepLearning/backup

## Run the training

At last let's run the training !

You will run it for 6000 iterations. This will take approximately one hour.
Every 100 iteration, the darknet framework will display the mAP (mean average precision) which is a measure of how well the network recognise the objects.
Eventually you will see lines like:

`mean average precision (mAP@0.50) = 0.751946, or 75.19 %`

(here 75.19% is already a very good mAP). Usually the mAP will increase during the training. If it stop increasing, it's time to stop the training.
During the training, the computed weights are periodically saved to the `NAO_DeepLearning/backup` folder we created in your google drive.

In [ ]:
# Run the training on our dataset, this takes about 1 hour when using a Colab 
# with a GPU
!./darknet detector train data/obj.data cfg/yolov4-tiny-tutorial.cfg cfg/yolov4-tiny.conv.29 -map -dont_show 2>&1 | tee backup/darknet.log | grep -vE '(^v3|Loaded|Tensor|^$|Last accuracy|next mAP)'

## Validate our model

You can validate that your model has been correctly trained by running it on images it has never seen. For this purpose we previously downloaded 80 images (40 for each objects) that we did not use during the training. Let's run our model on it and calculate the mAP (mean average precision).

First put the images in the correct folder, and resize them.

In [ ]:
%mkdir -p data/valid/
%cp -r OIDv4_ToolKit/OID/Dataset/validation/* data/valid/
!find data/valid/ -name "*jpg" > data/test.txt
!mogrify -verbose -strip -resize 416x416! -quality 75 data/valid/**/*jpg


Then let's run our model on them.

It will output something like:

```
class_id = 0, name = Glasses, ap = 66.72%   	 (TP = 31, FP = 7) 
class_id = 1, name = Mobile phone, ap = 85.36%   	 (TP = 50, FP = 10) 

 IoU threshold = 50 %, used Area-Under-Curve for each unique Recall 
 mean average precision (mAP@0.50) = 0.760398, or 76.04 % 
```

If you chose different objects than Glasses and Mobile phone, it will use your objects names. We obtained the log above when we prepared this tutorial. In our case, our model had a mAP of 66.72% on Glasses, and 85.36% on Mobile Phone, and a global mAP of 76.04%. This is very good! 

How much do you obtain ? Find out with the code below.

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-tiny-tutorial.cfg backup/yolov4-tiny-tutorial_best.weights


## Gather all files in Google Drive

Finally gather all the required files you will need to upload to the NAO Deep Learning application.

First generate translations for class names in french, so that you can use the NAO app in French if you want.

In [ ]:
translations = {"Tortoise": "Tortue", "Container": "Récipient", "Magpie": "Pie", "Sea turtle": "Tortue de mer", "Football": "Football", "Ambulance": "Ambulance", "Ladder": "Échelle", "Toothbrush": "Brosse à dents", "Syringe": "Seringue", "Sink": "Évier", "Toy": "Jouet", "Organ": "Organe", "Cassette deck": "Platine cassette", "Apple": "Pomme", "Human eye": "Œil humain", "Cosmetics": "Produits de beauté", "Paddle": "Pagayer", "Snowman": "Bonhomme de neige", "Beer": "Bière", "Chopsticks": "Baguettes", "Human beard": "Barbe humaine", "Bird": "Oiseau", "Parking meter": "Parcmètre", "Traffic light": "Feu de circulation", "Croissant": "Croissant", "Cucumber": "Concombre", "Radish": "Un radis", "Towel": "Serviette", "Doll": "Poupée", "Skull": "Le crâne", "Washing machine": "Machine à laver", "Glove": "Gant", "Tick": "Cocher", "Belt": "Courroie", "Sunglasses": "Des lunettes de soleil", "Banjo": "Banjo", "Cart": "Chariot", "Ball": "Balle", "Backpack": "Sac à dos", "Bicycle": "Bicyclette", "Home appliance": "Électroménager", "Centipede": "Mille-Pattes", "Boat": "Bateau", "Surfboard": "Planche de surf", "Boot": "Démarrage", "Headphones": "Écouteurs", "Hot dog": "Hot-dog", "Shorts": "Shorts", "Fast food": "Fast food", "Bus": "Bus", "Boy": "Garçon", "Screwdriver": "Tournevis", "Bicycle wheel": "Roue de vélo", "Barge": "Barge", "Laptop": "Portable", "Miniskirt": "Mini jupe", "Drill": "Percer", "Dress": "Robe", "Bear": "Ours", "Waffle": "Gaufre", "Pancake": "Crêpe", "Brown bear": "ours brun", "Woodpecker": "Pivert", "Blue jay": "Geai bleu", "Pretzel": "Bretzel", "Bagel": "Beignet", "Tower": "La tour", "Teapot": "Théière", "Person": "Une personne", "Bow and arrow": "Arc et des flèches", "Swimwear": "Maillots de bain", "Beehive": "Ruche", "Brassiere": "Soutien-gorge", "Bee": "abeille", "Bat": "Chauve souris", "Starfish": "Étoile de mer", "Popcorn": "Pop corn", "Burrito": "Burrito", "Chainsaw": "Tronçonneuse", "Balloon": "Ballon", "Wrench": "Clé", "Tent": "Tente", "Vehicle registration plate": "Plaque d'immatriculation du véhicule", "Lantern": "Lanterne", "Toaster": "Grille-pain", "Flashlight": "Lampe de poche", "Billboard": "Panneau d'affichage", "Tiara": "Tiare", "Limousine": "Limousine", "Necklace": "Collier", "Carnivore": "Carnivore", "Scissors": "Les ciseaux", "Stairs": "Escaliers", "Computer keyboard": "Clavier d'ordinateur", "Printer": "Imprimante", "Traffic sign": "Panneau de signalisation", "Chair": "Chaise", "Shirt": "Chemise", "Poster": "Affiche", "Cheese": "Du fromage", "Sock": "Chaussette", "Fire hydrant": "Bouche d'incendie", "Land vehicle": "Véhicule terrestre", "Earrings": "Des boucles d'oreilles", "Tie": "Attacher", "Watercraft": "Motomarine", "Cabinetry": "Ébénisterie", "Suitcase": "Valise", "Muffin": "Muffin", "Bidet": "Bidet", "Snack": "Goûter", "Snowmobile": "Motoneige", "Clock": "L'horloge", "Medical equipment": "Équipement médical", "Cattle": "Bétail", "Cello": "Violoncelle", "Jet ski": "Jet ski", "Camel": "chameau", "Coat": "Manteau", "Suit": "Combinaison", "Desk": "Pupitre", "Cat": "Chat", "Bronze sculpture": "Sculpture en bronze", "Juice": "Jus", "Gondola": "Gondole", "Beetle": "Scarabée", "Cannon": "Canon", "Computer mouse": "Souris d'ordinateur", "Cookie": "Biscuit", "Office building": "Immeuble de bureaux", "Fountain": "Fontaine", "Coin": "Pièce de monnaie", "Calculator": "Calculatrice", "Cocktail": "Cocktail", "Computer monitor": "Moniteur d'ordinateur", "Box": "Boîte", "Stapler": "Agrafeuse", "Christmas tree": "Sapin de Noël", "Cowboy hat": "Chapeau de cowboy", "Hiking equipment": "Matériel de randonnée", "Studio couch": "Canapé Studio", "Drum": "Tambouriner", "Dessert": "Dessert", "Wine rack": "Casier à vin", "Drink": "Boire", "Zucchini": "Courgette", "Ladle": "Louche", "Human mouth": "Bouche humaine", "Dairy": "Laitier", "Dice": "Dé", "Oven": "Four", "Dinosaur": "Dinosaure", "Ratchet": "Rochet", "Couch": "Canapé", "Cricket ball": "Balle de cricket", "Winter melon": "Melon d'hiver", "Spatula": "Spatule", "Whiteboard": "Tableau blanc", "Pencil sharpener": "Taille-crayon", "Door": "Porte", "Hat": "Chapeau", "Shower": "Douche", "Eraser": "La gomme", "Fedora": "Feutre", "Guacamole": "Guacamole", "Dagger": "Dague", "Scarf": "Foulard", "Dolphin": "Dauphin", "Sombrero": "Sombrero", "Tin can": "Boîte de conserve", "Mug": "Agresser", "Tap": "Robinet", "Harbor seal": "Phoque commun", "Stretcher": "Tendeur", "Can opener": "Ouvre-boîte", "Goggles": "Des lunettes de protection", "Human body": "Corps humain", "Roller skates": "Patins à roulettes", "Coffee cup": "Tasse à café", "Cutting board": "Planche à découper", "Blender": "Mixeur", "Plumbing fixture": "Appareil de plomberie", "Stop sign": "Panneau stop", "Office supplies": "Fournitures de bureau", "Volleyball": "Volley-ball", "Vase": "Vase", "Slow cooker": "Mijoteuse", "Wardrobe": "Penderie", "Coffee": "Café", "Whisk": "Fouet", "Paper towel": "Essuie-tout", "Personal care": "Soins personnels", "Food": "Aliments", "Sun hat": "chapeau de soleil", "Tree house": "Cabane dans les arbres", "Flying disc": "Disque volant", "Skirt": "Jupe", "Gas stove": "Cuisinière à gaz", "Salt and pepper shakers": "La salière et la poivrière", "Mechanical fan": "Ventilateur mécanique", "Face powder": "Poudre pour le visage", "Fax": "Fax", "Fruit": "Fruit", "French fries": "frites", "Nightstand": "La table de nuit", "Barrel": "Baril", "Kite": "cerf-volant", "Tart": "Tarte", "Treadmill": "Tapis roulant", "Fox": "Renard", "Flag": "Drapeau", "Horn": "Corne", "Window blind": "Store de fenêtre", "Human foot": "Pied humain", "Golf cart": "Voiturette de golf", "Jacket": "Veste", "Egg": "Œuf", "Street light": "éclairage public", "Guitar": "Guitare", "Pillow": "Oreiller", "Human leg": "Jambe humaine", "Isopod": "Isopode", "Grape": "Grain de raisin", "Human ear": "Oreille humaine", "Power plugs and sockets": "Fiches et prises de courant", "Panda": "Panda", "Giraffe": "Girafe", "Woman": "Femme", "Door handle": "Poignée de porte", "Rhinoceros": "Rhinocéros", "Bathtub": "Baignoire", "Goldfish": "Poisson rouge", "Houseplant": "Plante d'appartement", "Goat": "Chèvre", "Baseball bat": "Batte de baseball", "Baseball glove": "Gant de baseball", "Mixing bowl": "Bol mélangeur", "Marine invertebrates": "Invertébrés marins", "Kitchen utensil": "Ustensile de cuisine", "Light switch": "Interrupteur", "House": "loger", "Horse": "Cheval", "Stationary bicycle": "Vélo stationnaire", "Hammer": "Marteau", "Ceiling fan": "Ventilateur de plafond", "Sofa bed": "Canapé-lit", "Adhesive tape": "Ruban adhésif", "Harp": "Harpe", "Sandal": "Sandale", "Bicycle helmet": "Casque de vélo", "Saucer": "Soucoupe", "Harpsichord": "Clavecin", "Human hair": "Cheveux humains", "Heater": "Chauffe-eau", "Harmonica": "Harmonica", "Hamster": "Hamster", "Curtain": "Rideau", "Bed": "Lit", "Kettle": "Bouilloire", "Fireplace": "Cheminée", "Scale": "Escalader", "Drinking straw": "Paille", "Insect": "Insecte", "Hair dryer": "Sèche-cheveux", "Kitchenware": "Ustensiles de cuisine", "Indoor rower": "Rameur d'intérieur", "Invertebrate": "Invertébré", "Food processor": "Robot culinaire", "Bookcase": "Bibliothèque", "Refrigerator": "Réfrigérateur", "Wood-burning stove": "Poêle à bois", "Punching bag": "Sac de boxe", "Common fig": "Figue commune", "Cocktail shaker": "Shaker à cocktail", "Jaguar": "Jaguar", "Golf ball": "Balle de golf", "Fashion accessory": "Accessoire de mode", "Alarm clock": "Réveil", "Filing cabinet": "Classeur", "Artichoke": "Artichaut", "Table": "Table", "Tableware": "Vaisselle", "Kangaroo": "Kangourou", "Koala": "Koala", "Knife": "Couteau", "Bottle": "Bouteille", "Bottle opener": "Ouvre-bouteille", "Lynx": "Lynx", "Lavender": "Lavande", "Lighthouse": "Phare", "Dumbbell": "Haltère", "Human head": "Tête humaine", "Bowl": "bol", "Humidifier": "Humidificateur", "Porch": "Porche", "Lizard": "Lézard", "Billiard table": "Table de billard", "Mammal": "Mammifère", "Mouse": "Souris", "Motorcycle": "Moto", "Musical instrument": "Instrument de musique", "Swim cap": "Bonnet de bain", "Frying pan": "Poêle à frire", "Snowplow": "Chasse-neige", "Bathroom cabinet": "Armoire de toilette", "Missile": "Missile", "Bust": "Bousiller", "Man": "Homme", "Waffle iron": "Gaufrier", "Milk": "Lait", "Ring binder": "Reliure à anneaux", "Plate": "Plaque", "Mobile phone": "Téléphone mobile", "Baked goods": "Produits de boulangerie", "Mushroom": "Champignon", "Crutch": "Béquille", "Pitcher": "Lanceur", "Mirror": "Miroir", "Lifejacket": "Gilet de sauvetage", "Table tennis racket": "Raquette de tennis de table", "Pencil case": "Trousse", "Musical keyboard": "Clavier musical", "Scoreboard": "Tableau de bord", "Briefcase": "Mallette", "Kitchen knife": "Couteau de cuisine", "Nail": "Clou", "Tennis ball": "Balle de tennis", "Plastic bag": "Sac plastique", "Oboe": "Hautbois", "Chest of drawers": "Commode", "Ostrich": "Autruche", "Piano": "Piano", "Girl": "Fille", "Plant": "Usine", "Potato": "Pomme de terre", "Hair spray": "Laque pour les cheveux", "Sports equipment": "Équipement sportif", "Pasta": "Pâtes", "Penguin": "manchot", "Pumpkin": "Citrouille", "Pear": "Poire", "Infant bed": "Lit bébé", "Polar bear": "Ours polaire", "Mixer": "Mixer", "Cupboard": "Armoire", "Jacuzzi": "Jacuzzi", "Pizza": "Pizza", "Digital clock": "Horloge digitale", "Pig": "Cochon", "Reptile": "Reptile", "Rifle": "Fusil", "Lipstick": "Rouge à lèvres", "Skateboard": "planche à roulette", "Raven": "corbeau", "High heels": "Talons hauts", "Red panda": "Panda rouge", "Rose": "Rose", "Rabbit": "Lapin", "Sculpture": "Sculpture", "Saxophone": "Saxophone", "Shotgun": "Fusil à pompe", "Seafood": "Fruits de mer", "Submarine sandwich": "Sandwich sous-marin", "Snowboard": "Snowboard", "Sword": "Épée", "Picture frame": "Cadre de l'image", "Sushi": "Sushi", "Loveseat": "Causeuse", "Ski": "Ski", "Squirrel": "Écureuil", "Tripod": "Trépied", "Stethoscope": "Stéthoscope", "Submarine": "Sous-marin", "Scorpion": "Scorpion", "Segway": "Segway", "Training bench": "Banc d'entraînement", "Snake": "Serpent", "Coffee table": "Table basse", "Skyscraper": "Gratte-ciel", "Sheep": "Mouton", "Television": "Télévision", "Trombone": "Trombone", "Tea": "Thé", "Tank": "Réservoir", "Taco": "Taco", "Telephone": "Téléphone", "Torch": "Torche", "Tiger": "tigre", "Strawberry": "fraise", "Trumpet": "Trompette", "Tree": "Arbre", "Tomato": "Tomate", "Train": "Former", "Tool": "Outil", "Picnic basket": "Panier pique-nique", "Cooking spray": "Aérosol de cuisson", "Trousers": "Pantalon", "Bowling equipment": "Matériel de bowling", "Football helmet": "Casque de football américain", "Truck": "Camion", "Measuring cup": "Tasse à mesurer", "Coffeemaker": "Machine à café", "Violin": "Violon", "Vehicle": "Véhicule", "Handbag": "Sac à main", "Paper cutter": "Coupe-papier", "Wine": "Vin", "Weapon": "Arme", "Wheel": "Roue", "Worm": "Ver", "Wok": "Wok", "Whale": "Baleine", "Zebra": "Zèbre", "Auto part": "Pièce auto", "Jug": "Cruche", "Pizza cutter": "couteau à pizza", "Cream": "Crème", "Monkey": "Singe", "Lion": "Lion", "Bread": "Pain", "Platter": "Plat", "Chicken": "Poulet", "Eagle": "Aigle", "Helicopter": "Hélicoptère", "Owl": "Chouette", "Duck": "Canard", "Turtle": "Tortue", "Hippopotamus": "Hippopotame", "Crocodile": "Crocodile", "Toilet": "Toilette", "Toilet paper": "Papier toilette", "Squid": "Calmar", "Clothing": "Vêtements", "Footwear": "Chaussure", "Lemon": "Citron", "Spider": "Araignée", "Deer": "Cerf", "Frog": "La grenouille", "Banana": "Banane", "Rocket": "Fusée", "Wine glass": "Verre de vin", "Countertop": "Comptoir", "Tablet computer": "Tablette", "Waste container": "Conteneur à déchets", "Swimming pool": "Piscine", "Dog": "Chien", "Book": "Livre", "Elephant": "l'éléphant", "Shark": "Requin", "Candle": "Bougie", "Leopard": "Léopard", "Axe": "Hache", "Hand dryer": "Sèche mains", "Soap dispenser": "Distributeur de savon", "Porcupine": "Porc-épic", "Flower": "Fleur", "Canary": "Canari", "Cheetah": "guépard", "Palm tree": "palmier", "Hamburger": "Hamburger", "Maple": "Érable", "Building": "Bâtiment", "Fish": "Poisson", "Lobster": "Homard", "Asparagus": "Asperges", "Furniture": "Meubles", "Hedgehog": "Hérisson", "Airplane": "Avion", "Spoon": "Cuillère", "Otter": "loutre", "Bull": "Taureau", "Oyster": "huître", "Horizontal bar": "Barre horizontale", "Convenience store": "Épicerie", "Bomb": "Bombe", "Bench": "Banc", "Ice cream": "La crème glacée", "Caterpillar": "chenille", "Butterfly": "Papillon", "Parachute": "Parachute", "Orange": "Orange", "Antelope": "Antilope", "Beaker": "Gobelet", "Moths and butterflies": "Papillons et papillons", "Window": "La fenêtre", "Closet": "Toilettes", "Castle": "château", "Jellyfish": "Méduse", "Goose": "OIE", "Mule": "Mule", "Swan": "cygne", "Peach": "Pêche", "Coconut": "Noix de coco", "Seat belt": "Ceinture de sécurité", "Raccoon": "Raton laveur", "Chisel": "Ciseau", "Fork": "Fourchette", "Lamp": "Lampe", "Camera": "Caméra", "Squash": "Écraser", "Racket": "Raquette", "Human face": "Visage humain", "Human arm": "Bras humain", "Vegetable": "Légume", "Diaper": "Couche", "Unicycle": "Monocycle", "Falcon": "Faucon", "Chime": "Carillon", "Snail": "Escargot", "Shellfish": "Fruits de mer", "Cabbage": "Choux", "Carrot": "Carotte", "Mango": "Mangue", "Jeans": "jeans", "Flowerpot": "Pot de fleur", "Pineapple": "L'ananas", "Drawer": "Tiroir", "Stool": "Tabouret", "Envelope": "Enveloppe", "Cake": "Gâteau", "Dragonfly": "Libellule", "Sunflower": "Tournesol", "Microwave oven": "Four micro-onde", "Honeycomb": "Rayon de miel", "Marine mammal": "Mammifère marin", "Sea lion": "Lion de mer", "Ladybug": "Coccinelle", "Shelf": "Étagère", "Watch": "Regardez", "Candy": "Bonbons", "Salad": "salade", "Parrot": "Perroquet", "Handgun": "Pistolet", "Sparrow": "moineau", "Van": "Van", "Grinder": "Broyeur", "Spice rack": "étagère à épices", "Light bulb": "Ampoule", "Corded phone": "Téléphone filaire", "Sports uniform": "Uniforme de sport", "Tennis racket": "Raquette de tennis", "Wall clock": "horloge murale", "Serving tray": "Plateau", "Kitchen & dining room table": "Table de cuisine et salle à manger", "Dog bed": "Lit pour chien", "Cake stand": "Stand de gâteaux", "Cat furniture": "Meubles pour chats", "Bathroom accessory": "Accessoire de salle de bain", "Facial tissue holder": "Porte-mouchoirs en papier", "Pressure cooker": "Cocotte minute", "Kitchen appliance": "Appareil de cuisine", "Tire": "Pneu", "Ruler": "Règle", "Luggage and bags": "Bagages et sacs", "Microphone": "Microphone", "Broccoli": "Brocoli", "Umbrella": "Parapluie", "Pastry": "Pâtisserie", "Grapefruit": "Pamplemousse", "Band-aid": "Pansement", "Animal": "Animal", "Bell pepper": "poivron", "Turkey": "dinde", "Lily": "Lis", "Pomegranate": "Grenade", "Doughnut": "Donut", "Glasses": "Lunettes", "Human nose": "Nez humain", "Pen": "Stylo", "Ant": "Fourmi", "Car": "Auto", "Aircraft": "Avion", "Human hand": "Main humaine", "Skunk": "Moufette", "Teddy bear": "ours en peluche", "Watermelon": "Pastèque", "Cantaloupe": "Cantaloup", "Dishwasher": "Lave-vaisselle", "Flute": "Flûte", "Balance beam": "Poutre d'équilibre", "Sandwich": "Sandwich", "Shrimp": "Crevette", "Sewing machine": "Machine à coudre", "Binoculars": "Jumelles", "Rays and skates": "Rayons et patins", "Ipod": "Ipod", "Accordion": "Accordéon", "Willow": "saule", "Crab": "Crabe", "Crown": "couronner", "Seahorse": "hippocampe", "Perfume": "Parfum", "Alpaca": "Alpaga", "Taxi": "Taxi", "Canoe": "Canoë", "Remote control": "Télécommande", "Wheelchair": "Fauteuil roulant", "Rugby ball": "Ballon de rugby", "Armadillo": "Tatou", "Maracas": "Maracas", "Helmet": "Casque"}
with open("data/obj.names") as file:
  lines = file.readlines()
  with open("data/obj.names.fr", "w") as fr:
    for line in lines:
      fr.write("%s\n" % translations[line.rstrip()])

Then copy all result files to folder `NAO_DeepLearning/backup/result`

In [ ]:
%mkdir -p backup/result/
%cp backup/yolov4-tiny-tutorial_best.weights backup/result/model.weights 
%cp cfg/yolov4-tiny-tutorial.cfg backup/result/model.cfg
%cp data/obj.names backup/result/objects.names.en
%cp data/obj.names.fr backup/result/objects.names.fr

And that's it ! You can go to your Google Drive `NAO_DeepLearning/backup/result` folder and download its content. You will have to upload it to the NAO Deep Learning application.